# Análise de Jogos da NBA
Página para explorar o histórico de jogos, filtrando por partida e visualizando os pontos dos times da casa e visitante.

In [ ]:
import pandas as pd
import panel as pn
pn.extension('plotly')
# Carregar dados
game = pd.read_csv('csv/game.csv', parse_dates=['game_date'])
team = pd.read_csv('csv/team.csv')
team.columns = [c.lower() for c in team.columns]
game.columns = [c.lower() for c in game.columns]
# Função para obter o nickname do time pelo id
def get_team_nickname(team_id):
    row = team[team['id'] == team_id]
    if not row.empty:
        return row.iloc[0]['nickname']
    return ''
# Criar coluna com nome do jogo
game['nome_jogo'] = game.apply(lambda row: f'{get_team_nickname(row["team_id_home"])} vs. {get_team_nickname(row["team_id_away"])} ({row["game_date"].strftime("%d/%m/%Y")})', axis=1)

In [ ]:
# Filtro de data e jogos
import datetime

data_min = game['game_date'].min()
data_max = game['game_date'].max()
start_default = max(pd.Timestamp('2020-01-01'), data_min)
end_default = data_max

date_range_slider = pn.widgets.DateRangeSlider(
    name='Intervalo de Data dos Jogos',
    start=data_min,
    end=data_max,
    value=(start_default, end_default),
    format='%d/%m/%Y',
    width=500
)

def jogos_filtrados():
    start, end = date_range_slider.value
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    mask = (game['game_date'] >= start) & (game['game_date'] <= end)
    return game.loc[mask, 'nome_jogo'].tolist()

jogo_selector = pn.widgets.Select(
    name='Filtrar por Jogo',
    options=jogos_filtrados(),
    value=jogos_filtrados()[0] if jogos_filtrados() else None,
    width=400
)

def atualizar_jogos(event=None):
    opcoes = jogos_filtrados()
    jogo_selector.options = opcoes
    if opcoes:
        jogo_selector.value = opcoes[0]
    else:
        jogo_selector.value = None

date_range_slider.param.watch(atualizar_jogos, 'value')


In [ ]:
def get_jogo_info(nome_jogo):
    row = game[game['nome_jogo'] == nome_jogo].iloc[0]
    home_team = get_team_nickname(row['team_id_home'])
    away_team = get_team_nickname(row['team_id_away'])
    pts_home = row['pts_home']
    pts_away = row['pts_away']
    total_pontos = pts_home + pts_away
    return home_team, away_team, pts_home, pts_away, total_pontos

In [ ]:
def painel_jogo(nome_jogo):
    home_team, away_team, pts_home, pts_away, total_pontos = get_jogo_info(nome_jogo)
    style_card = """
        background:rgba(0,30,60,0.7);
        border-radius:32px;
        padding:32px 16px;
        color:#fff;
        box-shadow:0 4px 24px #0006;
        text-align:center;
        font-family:sans-serif;
        height:150px;
        display:flex;
        flex-direction:column;
        justify-content:center;
        align-items:center;
        width:250px;
    """
    style_title = 'font-size:2.5em;font-weight:bold;text-shadow:0 0 8px #00f;'
    style_label = 'font-size:1.2em;margin-bottom:8px;'
    style_points = 'font-size:2.5em;font-weight:bold;'
    style_total = 'font-size:2em;font-weight:bold;'
    pn.extension(raw_css=[
        """
        .jogo-row {
            background: linear-gradient(135deg, rgba(0,30,60,0.6), rgba(0,20,40,0.4));
            padding: 18px;
            border-radius: 20px;
            box-shadow: 0 8px 30px rgba(0,0,0,0.5);
            align-items: start;
        }
        .jogo-row .bk-root .bk-layout-cell {
            padding: 8px;
        }
        """
    ])

    return pn.Column(
        pn.Spacer(height=20),
        pn.Row(
            pn.Column(
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time da Casa</div><div style="{style_title}">{home_team}</div></div>'),
                pn.Spacer(height=20),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_home:,.2f}</div></div>'),
            ),
            pn.Column(
                pn.Spacer(height=40),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Total de Pontos</div><div style="{style_total}">{total_pontos:,.0f}</div></div>'),
            ),
            pn.Column(
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Time Visitante</div><div style="{style_title}">{away_team}</div></div>'),
                pn.Spacer(height=20),
                pn.pane.Markdown(f'<div style="{style_card}"><div style="{style_label}">Pontos</div><div style="{style_points}">{pts_away:,.2f}</div></div>'),
            ),
            css_classes=['jogo-row'],
        ),
        pn.Spacer(height=20),
        pn.Row(     
            pn.Spacer(width=100),
            pn.Column(
                pn.pane.Markdown('<div style="text-align:center;font-size:1.2em;color:#fff;">Intervalo de Data dos Jogos</div>'),
                date_range_slider,
                pn.pane.Markdown('<div style="text-align:center;font-size:1.2em;color:#fff;">Filtrar por Jogo</div>'),
                jogo_selector,
            ),
            pn.Spacer(width=100),
        ),
    )


In [ ]:
game_analysis_dashboard = pn.bind(painel_jogo, jogo_selector)
game_analysis_dashboard